In [1]:
# ==============================
# 2D Heat Conduction (CN + Picard)
# ==============================
import numpy as np
from dataclasses import dataclass
from typing import Callable, Tuple, Literal, Optional

try:
    import scipy.sparse as sp
    import scipy.sparse.linalg as spla
    _HAVE_SCIPY = True
except Exception:
    _HAVE_SCIPY = False

BCType = Literal["Dirichlet", "Neumann", "Robin"]  # Robin: convection (flux into domain)

# ---- Problem definitions ----
@dataclass
class Material2D:
    rho: Callable[[np.ndarray, np.ndarray, float, np.ndarray], np.ndarray]  # rho(X,Y,t,T)
    cp:  Callable[[np.ndarray, np.ndarray, float, np.ndarray], np.ndarray]  # cp(X,Y,t,T)
    k:   Callable[[np.ndarray, np.ndarray, float, np.ndarray], np.ndarray]  # k(X,Y,t,T) isotropic

@dataclass
class Source2D:
    qdot: Callable[[np.ndarray, np.ndarray, float], np.ndarray]             # qdot(X,Y,t) [W/m^3]

@dataclass
class BC2D:
    # For left/right: functions of (y, t); for bottom/top: functions of (x, t)
    # Return one of:
    #   ("Dirichlet", T_array)
    #   ("Neumann", q_array)           # positive INTO domain
    #   ("Robin", (h_array, Tinf_array))
    left:   Callable[[np.ndarray, float], Tuple[BCType, object]]
    right:  Callable[[np.ndarray, float], Tuple[BCType, object]]
    bottom: Callable[[np.ndarray, float], Tuple[BCType, object]]
    top:    Callable[[np.ndarray, float], Tuple[BCType, object]]

@dataclass
class Grid2D:
    Lx: float
    Ly: float
    Nx: int
    Ny: int

    def __post_init__(self):
        assert self.Nx >= 2 and self.Ny >= 2, "Need at least 2 nodes in each direction"
        self.x = np.linspace(0.0, self.Lx, self.Nx)
        self.y = np.linspace(0.0, self.Ly, self.Ny)
        self.dx = self.x[1] - self.x[0]
        self.dy = self.y[1] - self.y[0]
        self.X, self.Y = np.meshgrid(self.x, self.y, indexing="ij")  # (Nx, Ny)

# ---- helpers ----
def _harm(a, b, eps=1e-20):
    return 2.0 * a * b / np.maximum(a + b, eps)

def _flatten_idx(i, j, Ny):
    return i * Ny + j

# ---- Solver ----
class Heat2DSolver:
    r"""
    rho(x,y,t,T)*cp(x,y,t,T) dT/dt = ∂/∂x(k ∂T/∂x) + ∂/∂y(k ∂T/∂y) + qdot(x,y,t)

    - Uniform grid, conservative finite-volume 5-point stencil
    - Crank–Nicolson (theta) in time, Picard for T-dependent properties
    - BCs on each side: Dirichlet, Neumann (flux INTO domain), Robin (convection)
    """

    def __init__(
        self,
        grid: Grid2D,
        material: Material2D,
        source: Source2D,
        bc: BC2D,
        theta: float = 0.5,            # 0.5=Crank–Nicolson, 1=implicit Euler
        max_iter: int = 15,
        tol: float = 1e-8
    ):
        assert 0.0 < theta <= 1.0
        self.g = grid
        self.mat = material
        self.src = source
        self.bc = bc
        self.theta = theta
        self.max_iter = max_iter
        self.tol = tol

    # --- assemble and solve A T^{n+1} = d (Picard linearization) ---
    def _assemble(self, Tn: np.ndarray, dt: float, t_n: float, t_np1: float):
        Nx, Ny = self.g.Nx, self.g.Ny
        dx, dy = self.g.dx, self.g.dy
        X, Y = self.g.X, self.g.Y
        th = self.theta

        # properties at t^n and predictor at t^{n+1}
        rho_n = self.mat.rho(X, Y, t_n, Tn)
        cp_n  = self.mat.cp(X, Y, t_n, Tn)
        k_n   = self.mat.k (X, Y, t_n, Tn)

        # explicit face diffusivities at n
        De_n = _harm(k_n[:-1, :], k_n[1:, :]) / (dx*dx)      # (Nx-1, Ny)
        Dn_n = _harm(k_n[:, :-1], k_n[:, 1:]) / (dy*dy)      # (Nx, Ny-1)

        # explicit operator L(Tn)
        L_n = np.zeros_like(Tn)
        # east/west
        L_n[1:-1, :] += De_n[1:, :] * (Tn[2:, :] - Tn[1:-1, :])
        L_n[1:-1, :] += -De_n[:-1, :] * (Tn[1:-1, :] - Tn[:-2, :])
        # north/south
        L_n[:, 1:-1] += Dn_n[:, 1:] * (Tn[:, 2:] - Tn[:, 1:-1])
        L_n[:, 1:-1] += -Dn_n[:, :-1] * (Tn[:, 1:-1] - Tn[:, :-2])

        q_n = self.src.qdot(X, Y, t_n)

        # Picard iterations
        Tstar = Tn.copy()
        for _ in range(self.max_iter):
            rho_np1 = self.mat.rho(X, Y, t_np1, Tstar)
            cp_np1  = self.mat.cp (X, Y, t_np1, Tstar)
            k_np1   = self.mat.k  (X, Y, t_np1, Tstar)

            Cn   = rho_n   * cp_n
            Cnp1 = rho_np1 * cp_np1

            # implicit face diffusivities at n+1
            De_np1 = _harm(k_np1[:-1, :], k_np1[1:, :]) / (dx*dx)
            Dn_np1 = _harm(k_np1[:, :-1], k_np1[:, 1:]) / (dy*dy)

            # sparse assembly (COO)
            rows, cols, data = [], [], []
            d = np.zeros(Nx*Ny)

            # boundary data at n and n+1
            y_edge = self.g.y
            x_edge = self.g.x

            bL_type, bL_np1 = self.bc.left (y_edge, t_np1)
            bR_type, bR_np1 = self.bc.right(y_edge, t_np1)
            bB_type, bB_np1 = self.bc.bottom(x_edge, t_np1)
            bT_type, bT_np1 = self.bc.top   (x_edge, t_np1)

            # at t^n (explicit parts)
            _, bL_n = self.bc.left (y_edge, t_n)
            _, bR_n = self.bc.right(y_edge, t_n)
            _, bB_n = self.bc.bottom(x_edge, t_n)
            _, bT_n = self.bc.top   (x_edge, t_n)

            # helpers to broadcast
            def _arr(v, N):
                v = np.asarray(v)
                if v.ndim == 0: v = np.full(N, float(v))
                return v

            # pre-broadcast Robin tuples
            if bL_type == "Robin":
                hL_np1, TinfL_np1 = bL_np1
                hL_np1, TinfL_np1 = _arr(hL_np1, Ny), _arr(TinfL_np1, Ny)
                hL_n, TinfL_n = bL_n
                hL_n, TinfL_n = _arr(hL_n, Ny), _arr(TinfL_n, Ny)
            if bR_type == "Robin":
                hR_np1, TinfR_np1 = bR_np1
                hR_np1, TinfR_np1 = _arr(hR_np1, Ny), _arr(TinfR_np1, Ny)
                hR_n, TinfR_n = bR_n
                hR_n, TinfR_n = _arr(hR_n, Ny), _arr(TinfR_n, Ny)
            if bB_type == "Robin":
                hB_np1, TinfB_np1 = bB_np1
                hB_np1, TinfB_np1 = _arr(hB_np1, Nx), _arr(TinfB_np1, Nx)
                hB_n, TinfB_n = bB_n
                hB_n, TinfB_n = _arr(hB_n, Nx), _arr(TinfB_n, Nx)
            if bT_type == "Robin":
                hT_np1, TinfT_np1 = bT_np1
                hT_np1, TinfT_np1 = _arr(hT_np1, Nx), _arr(TinfT_np1, Nx)
                hT_n, TinfT_n = bT_n
                hT_n, TinfT_n = _arr(hT_n, Nx), _arr(TinfT_n, Nx)

            # assemble every node
            for i in range(Nx):
                for j in range(Ny):
                    k = _flatten_idx(i, j, Ny)

                    # neighbor diffusivities (n+1)
                    De = De_np1[i, j]     if i < Nx-1 else 0.0
                    Dw = De_np1[i-1, j]   if i > 0    else 0.0
                    Dn = Dn_np1[i, j]     if j < Ny-1 else 0.0
                    Ds = Dn_np1[i, j-1]   if j > 0    else 0.0

                    # diagonal & neighbors (start with identity)
                    diag = 1.0
                    rhs  = Tn[i, j]
                    # explicit diffusion part
                    rhs += (1.0 - th) * dt * (L_n[i, j] / Cn[i, j])
                    # explicit source
                    rhs += dt * ((1.0 - th) * self.src.qdot(np.array([[X[i,j]]]), np.array([[Y[i,j]]]), t_n)[0,0] / Cn[i, j])

                    # implicit source
                    rhs += dt * (th * self.src.qdot(np.array([[X[i,j]]]), np.array([[Y[i,j]]]), t_np1)[0,0] / Cnp1[i, j])

                    # add implicit neighbor couplings
                    if i > 0:
                        val = - th * dt * Dw / Cnp1[i, j]
                        rows.append(k); cols.append(_flatten_idx(i-1, j, Ny)); data.append(val)
                        diag += th * dt * Dw / Cnp1[i, j]
                    if i < Nx-1:
                        val = - th * dt * De / Cnp1[i, j]
                        rows.append(k); cols.append(_flatten_idx(i+1, j, Ny)); data.append(val)
                        diag += th * dt * De / Cnp1[i, j]
                    if j > 0:
                        val = - th * dt * Ds / Cnp1[i, j]
                        rows.append(k); cols.append(_flatten_idx(i, j-1, Ny)); data.append(val)
                        diag += th * dt * Ds / Cnp1[i, j]
                    if j < Ny-1:
                        val = - th * dt * Dn / Cnp1[i, j]
                        rows.append(k); cols.append(_flatten_idx(i, j+1, Ny)); data.append(val)
                        diag += th * dt * Dn / Cnp1[i, j]

                    # handle boundaries (replace row if Dirichlet; otherwise add flux terms)
                    is_left   = (i == 0)
                    is_right  = (i == Nx - 1)
                    is_bottom = (j == 0)
                    is_top    = (j == Ny - 1)

                    # --- LEFT boundary ---
                    if is_left:
                        if bL_type == "Dirichlet":
                            # overwrite row with identity
                            rows.append(k); cols.append(k); data.append(1.0)
                            d[k] = np.asarray(bL_np1).ravel()[j]
                            continue
                        elif bL_type == "Neumann":
                            qL_np1 = _arr(bL_np1, Ny)[j]
                            qL_n   = _arr(bL_n,   Ny)[j]
                            rhs += dt * ( th * (2.0 * qL_np1 / (Cnp1[i,j] * dx)) + (1.0 - th) * (2.0 * qL_n / (Cn[i,j] * dx)) )
                        elif bL_type == "Robin":
                            h = hL_np1[j]; Tinf = TinfL_np1[j]
                            h_n = hL_n[j];  Tinf_n = TinfL_n[j]
                            diag += th * dt * (2.0 * h) / (Cnp1[i,j] * dx)
                            rhs += th * dt * (2.0 * h * Tinf) / (Cnp1[i,j] * dx)
                            rhs += (1.0 - th) * dt * (2.0 * h_n * (Tinf_n - Tn[i,j])) / (Cn[i,j] * dx)

                    # --- RIGHT boundary ---
                    if is_right:
                        if bR_type == "Dirichlet":
                            rows.append(k); cols.append(k); data.append(1.0)
                            d[k] = np.asarray(bR_np1).ravel()[j]
                            continue
                        elif bR_type == "Neumann":
                            qR_np1 = _arr(bR_np1, Ny)[j]
                            qR_n   = _arr(bR_n,   Ny)[j]
                            rhs += dt * ( th * (2.0 * qR_np1 / (Cnp1[i,j] * dx)) + (1.0 - th) * (2.0 * qR_n / (Cn[i,j] * dx)) )
                        elif bR_type == "Robin":
                            h = hR_np1[j]; Tinf = TinfR_np1[j]
                            h_n = hR_n[j];  Tinf_n = TinfR_n[j]
                            diag += th * dt * (2.0 * h) / (Cnp1[i,j] * dx)
                            rhs += th * dt * (2.0 * h * Tinf) / (Cnp1[i,j] * dx)
                            rhs += (1.0 - th) * dt * (2.0 * h_n * (Tinf_n - Tn[i,j])) / (Cn[i,j] * dx)

                    # --- BOTTOM boundary ---
                    if is_bottom:
                        if bB_type == "Dirichlet":
                            rows.append(k); cols.append(k); data.append(1.0)
                            d[k] = np.asarray(bB_np1).ravel()[i]
                            continue
                        elif bB_type == "Neumann":
                            qB_np1 = _arr(bB_np1, Nx)[i]
                            qB_n   = _arr(bB_n,   Nx)[i]
                            rhs += dt * ( th * (2.0 * qB_np1 / (Cnp1[i,j] * dy)) + (1.0 - th) * (2.0 * qB_n / (Cn[i,j] * dy)) )
                        elif bB_type == "Robin":
                            h = hB_np1[i]; Tinf = TinfB_np1[i]
                            h_n = hB_n[i];  Tinf_n = TinfB_n[i]
                            diag += th * dt * (2.0 * h) / (Cnp1[i,j] * dy)
                            rhs += th * dt * (2.0 * h * Tinf) / (Cnp1[i,j] * dy)
                            rhs += (1.0 - th) * dt * (2.0 * h_n * (Tinf_n - Tn[i,j])) / (Cn[i,j] * dy)

                    # --- TOP boundary ---
                    if is_top:
                        if bT_type == "Dirichlet":
                            rows.append(k); cols.append(k); data.append(1.0)
                            d[k] = np.asarray(bT_np1).ravel()[i]
                            continue
                        elif bT_type == "Neumann":
                            qT_np1 = _arr(bT_np1, Nx)[i]
                            qT_n   = _arr(bT_n,   Nx)[i]
                            rhs += dt * ( th * (2.0 * qT_np1 / (Cnp1[i,j] * dy)) + (1.0 - th) * (2.0 * qT_n / (Cn[i,j] * dy)) )
                        elif bT_type == "Robin":
                            h = hT_np1[i]; Tinf = TinfT_np1[i]
                            h_n = hT_n[i];  Tinf_n = TinfT_n[i]
                            diag += th * dt * (2.0 * h) / (Cnp1[i,j] * dy)
                            rhs += th * dt * (2.0 * h * Tinf) / (Cnp1[i,j] * dy)
                            rhs += (1.0 - th) * dt * (2.0 * h_n * (Tinf_n - Tn[i,j])) / (Cn[i,j] * dy)

                    # finalize row (if not Dirichlet which already continued)
                    rows.append(k); cols.append(k); data.append(diag)
                    d[k] = rhs

            # solve
            Ntot = Nx * Ny
            if _HAVE_SCIPY:
                A = sp.coo_matrix((data, (rows, cols)), shape=(Ntot, Ntot)).tocsr()
                Tvec = spla.spsolve(A, d)
            else:
                # fallback dense (okay for small grids)
                from numpy.linalg import solve as dense_solve
                A = np.zeros((Ntot, Ntot))
                for rr, cc, vv in zip(rows, cols, data):
                    A[rr, cc] += vv
                Tvec = dense_solve(A, d)

            Tnew = Tvec.reshape((Nx, Ny))
            if np.linalg.norm(Tnew - Tstar, ord=np.inf) < self.tol:
                return Tnew
            Tstar = Tnew

        # not strictly converged: return last iterate
        return Tstar

    def step(self, Tn: np.ndarray, dt: float, t_n: float):
        t_np1 = t_n + dt
        Tnp1 = self._assemble(Tn, dt, t_n, t_np1)
        return Tnp1, t_np1

    def run(self, T0: np.ndarray, t0: float, tf: float, dt: float,
            callback: Optional[Callable[[float, np.ndarray], None]] = None):
        T = T0.copy()
        t = t0
        while t < tf - 1e-15:
            dt_eff = min(dt, tf - t)
            T, t = self.step(T, dt_eff, t)
            if callback is not None:
                callback(t, T)
        return T, t





In [ ]:
# ---------------------------
# Minimal example / template
# ---------------------------
if __name__ == "__main__":
    # Domain and grid
    Lx, Ly = 0.01, 0.002      # 10 mm x 2 mm thin plate
    Nx, Ny = 81, 21
    grid = Grid2D(Lx=Lx, Ly=Ly, Nx=Nx, Ny=Ny)

    # Material (constant here; can be T/x/y/t dependent)
    k0, rho0, cp0 = 0.26, 1300.0, 2000.0
    def rho_fn(X, Y, t, T): return np.full_like(X, rho0)
    def cp_fn (X, Y, t, T): return np.full_like(X, cp0)
    def k_fn  (X, Y, t, T): return np.full_like(X, k0)
    material = Material2D(rho=rho_fn, cp=cp_fn, k=k_fn)

    # No internal heat generation
    def qdot_fn(X, Y, t): return np.zeros_like(X)
    source = Source2D(qdot=qdot_fn)

    # BCs: hot plates on left/right at 250 C, insulated top/bottom
    TL, TR = 250.0, 250.0
    def bc_left (y, t):  return "Dirichlet", np.full_like(y, TL, dtype=float)
    def bc_right(y, t):  return "Dirichlet", np.full_like(y, TR, dtype=float)
    def bc_bottom(x, t): return "Neumann",   np.zeros_like(x, dtype=float)  # adiabatic
    def bc_top   (x, t): return "Neumann",   np.zeros_like(x, dtype=float)  # adiabatic
    bc = BC2D(left=bc_left, right=bc_right, bottom=bc_bottom, top=bc_top)

    # Initial condition: 30 C
    T0 = np.full((Nx, Ny), 30.0, dtype=float)

    solver = Heat2DSolver(grid, material, source, bc, theta=0.5, max_iter=20, tol=1e-9)

    # time stepping
    alpha = k0/(rho0*cp0)
    dt = 0.25 * min(grid.dx**2, grid.dy**2) / alpha  # accuracy-oriented
    t0, tf = 0.0, 1200.0

    # track centerline temperature
    i_mid = Nx//2
    T_end, _ = solver.run(T0, t0, tf, dt,
                          callback=lambda t, T: None)

    print("Done. Center temperature =", T_end[i_mid, Ny//2])